# **Predictive Model for Motor Vehicle Accidents: Injury and Fatality Prevention and Intersection Safety  (January 2015 - December 2024)**

#### *By Alejandro J. Ordonez*

**Version 1.0.1**

## **Project Overview**:

This project outlines the motor vehicle (cars, trucks, buses, motorcycle) accident data in NYC, based on intersection characteristics, finding patterns within drivers, accident concentration by zip code, and commonalities among high risk areas and vehicles. 

##### **Our Goal**: 

This project aims to create a model to predict potential injury or fatalities based on variables given by NYPD public data. It also aims to create an interactive heat map for intersection collisions and provide a comprehensive look at where collisions are occuring in NYC.

Further exploration of this model could aid insurance companies in adjusting premiums for high risk profiles and locations, provide government agencies with actionable data to improve traffic safety, and give new yorkers a compreshensive look at where accidents are occuring. 

We plan on doing so by analyzing historical accident data from the last 10 years, traffic patterns, and intersection structures. The predictive model will identify high risk areas and attempt to predict future accidents within a certain margin of error. 

##### **Methodology**: 

We plan on using NYC public data in order to create this model.

We will be using SQL, and more specifically Google's RDBMS BigQuery to retrieve and clean data. 

We will be using R to help with statistical analysis on the cleaned dataset, as well as to create the logistical regression model for injury/fatality analysis. 

And finally we will be using Tableau to create a heat map of high risk areas and intersections with high counts of collisions. 

# **Data Collection:**

To get our dataset we looked on the NYC public data in order to find the motor vehicle accident data. 

We used this table for the personal profiles: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu/about_data

We used this table for the collision in the intersection: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data

We used this table for the specific vehicles in the crashes: https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4/about_data

All of the data that we have collected will be stored into a personal Google Cloud and we will use Google's BigQuery in order to query the data. 

**For this project we will refer to the dataset as `insurance` and the tables as `insurance.persons`, `insurance.crashes`, and `insurance.vehicles` respectively.**

# **Data Cleaning**

As of now, SQL queries in this notebook do not run, however the final product is in the `motor-vehicle-accident-model` repo under the csv: `clean_motor_vehicle_table`. The following steps document the data cleaning that created the CSV we will be using. 

Using BigQuery, we will first clean the first table, `insurance.persons`. We will filter out null values for relevant fields and select data in between 2015-2024. 

In [ ]:
SELECT *
FROM 
  `insurance.persons`
WHERE
  `CRASH_DATE` BETWEEN '2015-01-01' AND '2024-12-01'
  AND PERSON_TYPE IS NOT NULL
  AND PERSON_AGE IS NOT NULL 
  AND PERSON_AGE >0 
  AND PERSON_AGE <150
  AND PERSON_SEX IS NOT NULL
  AND COLLISION_ID IS NOT NULL
  AND PERSON_TYPE = "Occupant"
  AND EMOTIONAL_STATUS IS NOT NULL
  AND POSITION_IN_VEHICLE LIKE "%Driver%"
  AND PERSON_INJURY IS NOT NULL

Now we will move onto the next table `insurance.crashes` where we will do largely the same thing as we did for the previous table. 

In [ ]:
  SELECT *
  FROM `insurance.crashes`
  WHERE
    `CRASH DATE` BETWEEN '2015-01-01' AND '2024-12-01'
    AND BOROUGH IS NOT NULL
    AND `ZIP CODE` IS NOT NULL
    AND `ZIP CODE` > 10000
    AND COLLISION_ID IS NOT NULL
    AND `NUMBER OF PERSONS KILLED` IS NOT NULL
    AND `NUMBER OF PERSONS INJURED` IS NOT NULL

For the last table (`insurance.vehicles`) we need to condense down the types of vehicles from over 1000 into 11 types:

(Sedan, SUV, Taxi, Pickup Truck, Van, Emergency Vehicle, Livery, Bus, Motorcycle, Bicycle, Truck, and Other)

Like the other queries, we will also filter out null values for relevant fields and select data in between 2015-2024

In [ ]:
WITH cleaned_vehicles AS (
    SELECT 
        COLLISION_ID,
        DRIVER_LICENSE_STATUS,
        UPPER(TRIM(VEHICLE_TYPE)) AS VEHICLE_TYPE,
        DRIVER_SEX,
        
        --This case query will sort the type of vehicles (ex: motorcycles, trucks, sedans, SUVs) together
        CASE 
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SEDAN%' THEN 'Sedan'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SPORT UTILITY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SUV%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%STATION WAGON%' THEN 'SUV'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TAXI%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%CAB%' THEN 'Taxi'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%PICK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRUCK%' THEN 'Pickup Truck'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%VAN%' THEN 'Van'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%AMBULANCE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FIRE TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FDNY%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%POLICE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EMERGENCY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EMS%' THEN 'Emergency Vehicle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%LIVERY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%LIMO%' THEN 'Livery'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%BUS%' THEN 'Bus'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOTORCYCLE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SCOOTER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOPED%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-SCOOTER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOTORBIKE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-BIKE%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EBIKE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%DIRT BIKE%' THEN 'Motorcycle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%BICYCLE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-BIKE%' THEN 'Bicycle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%DUMP TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FLATBED%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRACTOR TRAILER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MAIL TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%GARBAGE%' THEN 'Truck'
            ELSE 'Other'

    END AS TYPE_OF_VEHICLE


    FROM 
        `insurance.vehicles`
    WHERE 
        `CRASH_DATE` BETWEEN '2015-01-01' AND '2024-12-01'
        AND VEHICLE_TYPE IS NOT NULL
        AND DRIVER_SEX IS NOT NULL
        AND DRIVER_LICENSE_STATUS IS NOT NULL
        AND COLLISION_ID IS NOT NULL

)

SELECT 
    DRIVER_LICENSE_STATUS,
    TYPE_OF_VEHICLE,
    DRIVER_SEX,
    cleaned_vehicles.VEHICLE_TYPE
FROM cleaned_vehicles

In this case, we used a common table expression "cleaned_vehicles" to first create a table then run the query. We will use this same methodology to combine the rest of the cleaned queries into one query and use **INNER JOIN** to combine the tables together. 

### Choosing Variables

Now that we have the cleaned tables, we need to select what we will be pulling from the new joined table. Of the variables for our model, we will pull a few that are relevant and can be factored in R. We ended up with the following fields:

* Crash date
* Crash time
* Borough
* Zip code
* Number of fatalities caused by crash
* Number of persons injured by crash
* Driver injury
* Drivers license Status
* Vehicle type
* Age
* Sex

## **Final Model Query**

In [ ]:
--The following will be cleaning the data before joining them together

--Cleaned insurance.crashes query
WITH cleaned_crashes AS (
  SELECT *
  FROM `insurance.crashes`
  WHERE
    `CRASH DATE` BETWEEN '2015-01-01' AND '2024-12-01'
    AND BOROUGH IS NOT NULL
    AND `ZIP CODE` IS NOT NULL
    AND `ZIP CODE` > 10000
    AND COLLISION_ID IS NOT NULL
    AND `NUMBER OF PERSONS KILLED` IS NOT NULL
    AND `NUMBER OF PERSONS INJURED` IS NOT NULL

),

--Cleaned insurance.vehicles query
cleaned_vehicles AS (
    SELECT 
        COLLISION_ID,
        DRIVER_LICENSE_STATUS,
        UPPER(TRIM(VEHICLE_TYPE)) AS VEHICLE_TYPE,
        DRIVER_SEX,
        
        #This case query will sort the type of vehicles (ex: motorcycles, trucks, sedans, SUVs) together
        CASE 
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SEDAN%' THEN 'Sedan'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SPORT UTILITY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SUV%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%STATION WAGON%' THEN 'SUV'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TAXI%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%CAB%' THEN 'Taxi'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%PICK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRUCK%' THEN 'Pickup Truck'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%VAN%' THEN 'Van'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%AMBULANCE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FIRE TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FDNY%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%POLICE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EMERGENCY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EMS%' THEN 'Emergency Vehicle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%LIVERY%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%LIMO%' THEN 'Livery'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%BUS%' THEN 'Bus'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOTORCYCLE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%SCOOTER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOPED%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-SCOOTER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MOTORBIKE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-BIKE%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%EBIKE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%DIRT BIKE%' THEN 'Motorcycle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%BICYCLE%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%E-BIKE%' THEN 'Bicycle'
            WHEN UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%DUMP TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%FLATBED%' 
                OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%TRACTOR TRAILER%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%MAIL TRUCK%' OR UPPER(TRIM(VEHICLE_TYPE)) LIKE '%GARBAGE%' THEN 'Truck'
            ELSE 'Other'

    END AS TYPE_OF_VEHICLE


    FROM 
        `insurance.vehicles`
    WHERE 
        `CRASH_DATE` BETWEEN '2015-01-01' AND '2024-12-01'
        AND VEHICLE_TYPE IS NOT NULL
        AND DRIVER_SEX IS NOT NULL
        AND DRIVER_LICENSE_STATUS IS NOT NULL
        AND COLLISION_ID IS NOT NULL

),

--Cleaned insurance.persons query
cleaned_persons AS (
  SELECT *
  FROM `insurance.persons`
  WHERE
    `CRASH_DATE` BETWEEN '2015-01-01' AND '2024-12-01'
    AND PERSON_TYPE IS NOT NULL
    AND PERSON_AGE IS NOT NULL 
    AND PERSON_AGE >0 
    AND PERSON_AGE <150
    AND PERSON_SEX IS NOT NULL
    AND COLLISION_ID IS NOT NULL
    AND PERSON_TYPE = "Occupant"
    AND EMOTIONAL_STATUS IS NOT NULL
    AND POSITION_IN_VEHICLE LIKE "%Driver%"
    AND PERSON_INJURY IS NOT NULL
)

--Here we will begin the query with the cleaned data
SELECT 
  c.`CRASH DATE`,
  c.`CRASH TIME`,
  c.BOROUGH,
  c.`ZIP CODE`,
  c.`NUMBER OF PERSONS KILLED`,
  c.`NUMBER OF PERSONS INJURED`,
  c.`CONTRIBUTING FACTOR VEHICLE 1`,
  v.DRIVER_LICENSE_STATUS,
  v.TYPE_OF_VEHICLE,
  p.PERSON_INJURY,
  p.PERSON_AGE,
  p.PERSON_SEX,
  COUNT(*) AS TOTAL_INCIDENTS

--Here we will join the 3 tables together with COLLISION_ID, where we rename the tables as c for Crashes, p for Persons, and v for Vehicles
FROM 
  cleaned_crashes AS c
  INNER JOIN cleaned_vehicles AS v
    ON c.COLLISION_ID = v.COLLISION_ID
  INNER JOIN cleaned_persons AS p
    ON c.COLLISION_ID = p.COLLISION_ID


GROUP BY 
  c.`CRASH DATE`,
  c.`CRASH TIME`,
  c.BOROUGH,
  c.`ZIP CODE`,
  c.`NUMBER OF PERSONS KILLED`,
  c.`NUMBER OF PERSONS INJURED`,
  c.`CONTRIBUTING FACTOR VEHICLE 1`,
  v.DRIVER_LICENSE_STATUS,
  v.TYPE_OF_VEHICLE,
  p.PERSON_INJURY,
  p.PERSON_AGE,
  p.PERSON_SEX

We now have a cleaned dataset with all of the variables for the model. Although the code above does not work in Jupyter notebook, the result is in the `motor-vehicle-accident-model` repo in my GitHub under the csv file `clean_motor_vehicle_table`. Now we can begin with the analysis in R. 

# **Model Data Analysis**

Now we will use R to analyze the motor vehicle data we had just cleaned. First we will retrieve tidyverse and read the CSV file as "vehicle_data". 

In [18]:
#Optional, install all packages we will be using for this analysis:
install.packages("tidyverse")
install.packages("RCurl")
install.packages("forcats")
install.packages("broom")
install.packages("ResourceSelection")


The downloaded binary packages are in
	/var/folders/4b/lyn1g4ln7fz23d5nk1h9vprm0000gn/T//RtmpvxEfjA/downloaded_packages

The downloaded binary packages are in
	/var/folders/4b/lyn1g4ln7fz23d5nk1h9vprm0000gn/T//RtmpvxEfjA/downloaded_packages

The downloaded binary packages are in
	/var/folders/4b/lyn1g4ln7fz23d5nk1h9vprm0000gn/T//RtmpvxEfjA/downloaded_packages

The downloaded binary packages are in
	/var/folders/4b/lyn1g4ln7fz23d5nk1h9vprm0000gn/T//RtmpvxEfjA/downloaded_packages

The downloaded binary packages are in
	/var/folders/4b/lyn1g4ln7fz23d5nk1h9vprm0000gn/T//RtmpvxEfjA/downloaded_packages


In [24]:
library(tidyverse)
library(RCurl)
raw_file <- getURL("https://raw.githubusercontent.com/ajordonez/motor-vehicle-accident-model/refs/heads/main/clean_motor_vehicle_table.csv")
vehicle_data <- read.csv(text = raw_file)
head(vehicle_data)

,CRASH.DATE,CRASH.TIME,BOROUGH,ZIP.CODE,NUMBER.OF.PERSONS.KILLED,NUMBER.OF.PERSONS.INJURED,CONTRIBUTING.FACTOR.VEHICLE.1,DRIVER_LICENSE_STATUS,TYPE_OF_VEHICLE,PERSON_INJURY,PERSON_AGE,PERSON_SEX,TOTAL_INCIDENTS
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>
1,2/12/19,11:55,MANHATTAN,10001,0,0,Brakes Defective,Licensed,Sedan,Unspecified,31,M,2
2,3/13/17,14:21,MANHATTAN,10001,0,0,Following Too Closely,Licensed,Sedan,Unspecified,67,F,1
3,6/28/19,19:30,MANHATTAN,10001,0,0,Unsafe Lane Changing,Licensed,SUV,Unspecified,70,F,1
4,7/26/19,16:00,MANHATTAN,10001,0,0,Following Too Closely,Licensed,Sedan,Unspecified,78,M,1
5,8/15/17,11:12,MANHATTAN,10001,0,0,Following Too Closely,Licensed,Taxi,Unspecified,38,M,2
6,3/30/18,17:10,MANHATTAN,10001,0,0,Driver Inattention/Distraction,Licensed,Sedan,Unspecified,62,M,1


We will now convert the following fields into factors for easier analysis

In [25]:
vehicle_data$BOROUGH <- as.factor(vehicle_data$BOROUGH)
vehicle_data$BOROUGH <- relevel(factor(vehicle_data$BOROUGH), ref = "STATEN ISLAND")
vehicle_data$PERSON_INJURY <- as.factor(vehicle_data$PERSON_INJURY)
vehicle_data$PERSON_SEX <- as.factor(vehicle_data$PERSON_SEX)
vehicle_data$DRIVER_LICENSE_STATUS <- factor(vehicle_data$DRIVER_LICENSE_STATUS, 
                                           levels = c("Unlicensed", "Permit", "Licensed"), 
                                           ordered = TRUE)
vehicle_data$TYPE_OF_VEHICLE <- as.factor(vehicle_data$TYPE_OF_VEHICLE)

#Make sure that for drivers license status it is ordered
levels(vehicle_data$DRIVER_LICENSE_STATUS)


[1] "Unlicensed" "Permit"     "Licensed"

We now have most of the variables as factors, but we also have age as a factor for our model. We will now check to make sure for age, no values fall out of reasonable driver age.

In [26]:
max(vehicle_data$PERSON_AGE); min(vehicle_data$PERSON_AGE)

[1] 145

[1] 1

We can see that the minimum is 1 and maximum is 145. This is not realistic and was probably due to data entry error. We will make the minimum value 12 and maximum 100 for this analysis.


In [27]:
vehicle_data <- vehicle_data %>% filter(PERSON_AGE > 12, PERSON_AGE < 100)

For our injury/fatality analysis, we also need to make a seperate field so that we can check if the collision cause any injuries or fatalities.

In [28]:
vehicle_data <- vehicle_data %>%
  mutate(INJURY_OR_FATALITY = ifelse(PERSON_INJURY %in% c("Injured", "Killed"), TRUE, FALSE))

After a quick check on the count of TYPE_OF_VEHICLES with our newly created INJURY_OR_FATALITY column

In [29]:
table(vehicle_data$TYPE_OF_VEHICLE, vehicle_data$INJURY_OR_FATALITY)

                   
                    FALSE  TRUE
  Bus                1919   136
  Emergency Vehicle   371    26
  Livery               23     0
  Motorcycle          680   410
  Other              2182   125
  Pickup Truck       5392   375
  SUV               22633  2666
  Sedan             29209  3682
  Taxi               3792   277
  Truck               188    10
  Van                 863    62

We can see that ***Livery*** (Limo services) has less than 30 instances, meaning it might cause our model some errors as a variable. We will combine ***Livery*** with Taxi services for this analysis as they both functionally do similar things, drive a paying client/passenger from one place to another.

In [30]:
library(forcats)
vehicle_data <- vehicle_data %>%
  mutate(TYPE_OF_VEHICLE = fct_collapse(TYPE_OF_VEHICLE, "Taxi/Livery" = c("Livery", "Taxi")))

#### **NOTE: The following will show the processes to get the final model. If you would like to skip to the final model, run the code for the following steps and go to "Final Regression Model"**

Now it is time to make the first model, we will use a logistical regression model. We chose logistical regression as we are trying to find whether or not (True or False), the variables are factors for injury or fatality. 

In order to get the aggregrated intersections which have the most amount of accidents, we will have to dive deeper. The following query will clean, aggregate, and display the intersections with the most amount of collisions.

We will also have an additional column that displays the total number of collisions as well as getting the latitude and longitude for each collision, this will help us when displaying our results. 

In [6]:
SELECT 
  COUNT(DISTINCT COLLISION_ID) AS number_of_collisions, 
  LEAST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))) AS primary_street_name,
  GREATEST(TRIM(UPPER(`ON STREET NAME`)), TRIM(UPPER(`CROSS STREET NAME`))) AS secondary_street_name,
  LATITUDE as latitude,

WHERE

IndentationError: unexpected indent (2933826755.py, line 2)